**Implementierung Kollaboratives modelbasiertes System mit Matrixfaktorisierung **

*Installation Surprise Bibliothek*

In [ ]:
pip install scikit-surprise # Quelle 1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357257 sha256=e7765abf0df8db43041fea87eecc29e38dc52ba5f4b896fb4d6792816ba55b02
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


*Import benötigte Bibliotheken*


Importiert werden die für die Umsetzung benötigten Python Bibliotheken Pandas und Surprise

In [ ]:
import pandas as pd # Quelle 2
from surprise.model_selection import cross_validate # Quelle 3
from surprise import Dataset, Reader, accuracy, SVD, SVDpp, NMF, NormalPredictor # Quelle 3

*Datenvorverarbeitung*

Die Bewertungsdaten werden eingelesen und in ein Format überführt, mit welchem Surprise arbeiten kann.

In [ ]:
ratings = pd.read_csv('ratings.csv') # Quelle 4

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0)) # Quelle 5
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader) # Quelle 5

*Konfiguration der Ähnlichkeit Einstellungen*

Die Ähnlichkeits Konfigurationen werden erstellt und intitialisiert.

In [ ]:
#SVD Factors 150
algo_svd_150 = SVD(n_factors = 150)

*Evaluierungs Parameter: RMSE, MAE, MSE, FCP, Test-Time, Train-Time (SVD)*

Die Cross Validierung wird konfiguriert und durchgeführt.

In [ ]:
cross_validate(algo_svd_150, data, measures=["RMSE", "MAE", "MSE", "FCP"], cv=5, verbose=True) #Quelle 9

Evaluating RMSE, MAE, MSE, FCP of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8933  0.9018  0.8962  0.9012  0.8999  0.8985  0.0033  
MAE (testset)     0.6888  0.6937  0.6909  0.6960  0.6918  0.6922  0.0025  
MSE (testset)     0.7979  0.8132  0.8032  0.8122  0.8099  0.8073  0.0058  
FCP (testset)     0.6357  0.6404  0.6312  0.6392  0.6369  0.6367  0.0032  
Fit time          1.89    2.78    1.94    1.87    1.88    2.07    0.36    
Test time         0.12    0.38    0.15    0.24    0.12    0.20    0.10    


{'test_rmse': array([0.89326315, 0.9017934 , 0.89619941, 0.90120538, 0.89992982]),
 'test_mae': array([0.6887649 , 0.69365597, 0.6908878 , 0.69596747, 0.69177704]),
 'test_mse': array([0.79791905, 0.81323133, 0.80317338, 0.81217114, 0.80987369]),
 'test_fcp': array([0.63568685, 0.64042237, 0.63120465, 0.63923728, 0.63686711]),
 'fit_time': (1.8923990726470947,
  2.781872510910034,
  1.9436671733856201,
  1.8654751777648926,
  1.8808786869049072),
 'test_time': (0.11750459671020508,
  0.3771183490753174,
  0.147047758102417,
  0.2440495491027832,
  0.11597418785095215)}

*Generierung von Top N Empfehlungen*

In [ ]:
trainset = data.build_full_trainset() #Quelle 10
algo.fit(trainset) #Quelle 10

In [ ]:
def get_collaborative_filtering_recommendations(user_id, top_n):  #Quelle 11
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = algo.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations

In [ ]:
user_id = 10
top_n = 10
recommendations = get_collaborative_filtering_recommendations(user_id, top_n)

In [ ]:
print(f"Empfehlungen für Nutzer mit ID: {user_id}:") #Quelle 11
for i, recommendation in enumerate(recommendations):
    print(f"{i + 1}. Movie ID: {recommendation}")

Empfehlungen für Nutzer mit ID: 10:
1. Movie ID: 926
2. Movie ID: 1228
3. Movie ID: 1217
4. Movie ID: 969
5. Movie ID: 913
6. Movie ID: 2064
7. Movie ID: 1304
8. Movie ID: 858
9. Movie ID: 8132
10. Movie ID: 1254


Quellenverzeichnis:


Quelle 1: pypi.org, Installation scikit-surprise, letzter Aufruf: 18.07.2024,
Link:https://pypi.org/project/scikit-surprise/


Quelle 2: w3schools, Pandas Getting Started, letzter Aufruf: 18.07.2024,
Link:https://www.w3schools.com/python/pandas/pandas_getting_started.asp


Quelle 3: Read the Docs, Surprise Dokumentation, Getting started, letzter Aufruf: 18.07.2024 Link: https://surprise.readthedocs.io/en/stable/getting_started.html#getting-started

Quelle 4: w3schools, Pandas, letzter Aufruf: 18.07.2024, Link:https://www.w3schools.com/python/pandas/pandas_csv.asp

Quelle 5: Read the Docs, Surprise Dokumentation, Reader, letzter Aufruf: 18.07.2024, Link:https://surprise.readthedocs.io/en/stable/reader.html

Quelle 6: Read the Docs, Surprise Dokumentation, Ähnlichkeits Metriken konfigurieren, letzter Aufruf: 18.07.2024,
Link: https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration

Quelle 7: Read the Docs, Surprise Dokumentation, Ähnlichkeits Metriken Auswahl, letzter Aufruf: 18.07.2024, Link: https://surprise.readthedocs.io/en/stable/similarities.html

Quelle 8: Read the Docs, Surprise Dokumentation, Algorithmen Auswahl, letzter Aufruf: 18.07.2024, Link: https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

Quelle 9: Read the Docs, Surprise Dokumentation, Evaluationsverfahren, letzter Aufruf: 18.07.2024, Link: https://surprise.readthedocs.io/en/stable/accuracy.html

Quelle 10: Read the Docs, Surprise Dokumentation,letzter Aufruf: 18.07.2024, Link: https://surprise.readthedocs.io/en/stable/trainset.html

Quelle 11, AMAN KHARWAL, thecleverprogrammer, letzter Aufruf: 18.07.2024, Link: https://thecleverprogrammer.com/2023/06/05/hybrid-recommendation-system-using-python/